In [1]:
import os
import random
import pandas as pd
from collections import Counter
from itertools import product

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Make sure data folder exists
os.makedirs('../data', exist_ok=True)

# Generate clearly separated DNA sequences
def generate_extremely_biased_sequence(length, bias='human'):
    if bias == 'human':
        return ''.join(random.choices(['C', 'G'], k=length))  # only C and G
    else:
        return ''.join(random.choices(['A', 'T'], k=length))  # only A and T

# 1000 sequences: 500 human, 500 non-human
sequences = [generate_extremely_biased_sequence(100, 'human') if i < 500 else generate_extremely_biased_sequence(100, 'nonhuman') for i in range(1000)]
labels = [1 if i < 500 else 0 for i in range(1000)]

# Create dataframe and save
df = pd.DataFrame({'sequence': sequences, 'label': labels})
df.to_csv('../data/dna_dataset.csv', index=False)
df.head()


sequence  label
0  GCGCGCCCGCCGCCCGGCGCGCGGCCCCCCCGGGCCCCCCCCCGGG...      1
1  GGCCGCCCCCGGCCGCGCGCCCGGCCCGCCGGGGGGGCCCGGCCCC...      1
2  CGCGCCGCCGCGGCCCGCCGCCGGGCCGCGCGGCGGGGCGCGGGCC...      1
3  CCCCCGCCCGGGCGGCGGGCCCGGGGCCCGCCGCCGCCCGCCCCCC...      1
4  GCCGGCCCCGCGGCGGCGCGGGCCGCCCGGGGGCGCGCCGGCGGGC...      1

In [3]:
def get_kmers(sequence, k=3):
    return [sequence[i:i+k] for i in range(len(sequence)-k+1)]

def kmer_vector(sequence, k=3):
    kmers = get_kmers(sequence, k)
    count = Counter(kmers)
    all_kmers = [''.join(p) for p in product('ATCG', repeat=k)]
    return [count.get(kmer, 0) for kmer in all_kmers]

# Convert all sequences
X = df['sequence'].apply(lambda x: kmer_vector(x, 3))
X = pd.DataFrame(X.tolist())
y = df['label']
X.shape


(1000, 64)

In [4]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print("Train:", X_train.shape)
print("Test:", X_test.shape)

Train: (800, 64)
Test: (200, 64)


In [5]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(64,)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

Epoch 1/50


/opt/anaconda3/envs/dnaenv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7614 - loss: 0.5548 - val_accuracy: 1.0000 - val_loss: 0.1639
Epoch 2/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.1046 - val_accuracy: 1.0000 - val_loss: 0.0387
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0249 - val_accuracy: 1.0000 - val_loss: 0.0132
Epoch 4/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0095 - val_accuracy: 1.0000 - val_loss: 0.0067
Epoch 5/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0054 - val_accuracy: 1.0000 - val_loss: 0.0042
Epoch 6/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0034 - val_accuracy: 1.0000 - val_loss: 0.0027
Epoch 7/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0022 - val_accuracy: 1.0000 - val_loss: 0.0018
Epoch 8/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0015 - val_accuracy: 1.0000 - val_loss: 0.0012
Epo

In [6]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 2.1095e-05 
Test Accuracy: 1.0000
